In [1]:
! pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [3]:
from pyspark.sql import SparkSession

In [10]:
from pyspark.sql import SQLContext

spark.conf.set('spark.sql.repl.eagerEval.enabled', True) 

In [5]:
!hdfs dfs -ls

Found 42 items
drwxr-xr-x   - ubuntu hadoop          0 2024-02-23 08:37 .sparkStaging
-rw-r--r--   1 ubuntu hadoop 2807409271 2024-02-23 05:24 2019-08-22.txt
-rw-r--r--   1 ubuntu hadoop 2854479008 2024-02-23 05:26 2019-09-21.txt
-rw-r--r--   1 ubuntu hadoop 2895460543 2024-02-23 05:25 2019-10-21.txt
-rw-r--r--   1 ubuntu hadoop 2939120942 2024-02-23 05:25 2019-11-20.txt
-rw-r--r--   1 ubuntu hadoop 2995462277 2024-02-23 05:29 2019-12-20.txt
-rw-r--r--   1 ubuntu hadoop 2994906767 2024-02-23 05:29 2020-01-19.txt
-rw-r--r--   1 ubuntu hadoop 2995431240 2024-02-23 05:24 2020-02-18.txt
-rw-r--r--   1 ubuntu hadoop 2995176166 2024-02-23 05:24 2020-03-19.txt
-rw-r--r--   1 ubuntu hadoop 2996034632 2024-02-23 05:26 2020-04-18.txt
-rw-r--r--   1 ubuntu hadoop 2995666965 2024-02-23 05:25 2020-05-18.txt
-rw-r--r--   1 ubuntu hadoop 2994699401 2024-02-23 05:26 2020-06-17.txt
-rw-r--r--   1 ubuntu hadoop 2995810010 2024-02-23 05:27 2020-07-17.txt
-rw-r--r--   1 ubuntu hadoop 2995995152 2024-02-23

In [6]:
! hdfs dfs -ls /tmp

Found 3 items
drwxrwxrwt   - hdfs   hadoop          0 2024-02-23 05:06 /tmp/hadoop-yarn
drwxr-xr-x   - ubuntu hadoop          0 2024-02-23 07:48 /tmp/month_stat1
drwxr-xr-x   - ubuntu hadoop          0 2024-02-23 06:48 /tmp/month_stat2


In [7]:
# !hdfs dfs -put abalone_train.csv

In [8]:
#!ls .aws

In [9]:
spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "10g")
        .config("spark.driver.memory", "4g")
        .config("spark.executor.cores", "4")
        .config("spark.executor.instances", "3")
        .getOrCreate()
)

In [12]:
sql = SQLContext(spark)

In [13]:
sql

In [14]:
df = sql.read.parquet("s3a://yc-mdb-examples/dataproc/example01/set01")

In [16]:
defaultFS = spark._jsc.hadoopConfiguration().get("fs.defaultFS")
month_stat = df.groupBy("Month").count()
month_stat.repartition(1).write.format("csv").save(defaultFS+"/tmp/month_stat1")

In [18]:
data = (spark.read.format('csv')
        .options(header='true', inferSchema='true')
        .load('abalone_train.csv'))

In [19]:
data

DataFrame[0.435: double, 0.335: double, 0.11: double, 0.334: double, 0.1355: double, 0.0775: double, 0.0965: double, 7: int]

In [ ]:
data.show()

In [11]:
from pyspark.sql.types import *

schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("tx_datetime", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("terminal_id", IntegerType(), True),
    StructField("tx_amount", DoubleType(), True),
    StructField("tx_time_seconds", IntegerType(), True),
    StructField("tx_time_days", IntegerType(), True),
    StructField("tx_fraud", IntegerType(), True),
    StructField("tx_fraud_scenario", IntegerType(), True)
])

In [27]:
data = (
    spark.read.format('csv')
    .options(header='true', inferSchema='true', sep=','
             # , nullValue='?'
            )
    .load(['2019-08-22.txt', '2022-11-04.txt', '2022-10-05.txt'], schema=schema)
)

In [28]:
data

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0
1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0
2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0
3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0
4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0
5,2019-08-22 18:41:25,3,205,34.2,67285,0,0,0
6,2019-08-22 03:12:21,3,0,47.2,11541,0,0,0
7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0
8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0
9,2019-08-22 21:09:37,8,931,61.7,76177,0,0,0


In [23]:
data.show(5)

+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|             0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|             1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|             2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|             3|2019-08-22 07:21:33|          0|          0|    80.41|          26493|           0|       0|                0|
|             4|2019-08-22 09:06:17|          1|        981|   102.83|          32777|           0|       0|   

In [24]:
from pyspark.sql.functions import count, when, col

In [29]:
missing_data_counts = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

In [30]:
missing_data_counts

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario
0,0,0,2857,0,0,0,0,0


In [35]:
validity_check = data.filter("transaction_id >= 0 and tx_datetime not like '____-__-__ __:__:__'")

In [36]:
validity_check

transaction_id,tx_datetime,customer_id,terminal_id,tx_amount,tx_time_seconds,tx_time_days,tx_fraud,tx_fraud_scenario


In [39]:
num_rows_initial = data.count()
num_cols_initial = len(data.columns)

In [40]:
num_rows_initial, num_cols_initial

(140984588, 9)

In [ ]:
num_rows_validity_check = validity_check.count()
num_cols_validity_check = len(validity_check.columns)

num_rows_validity_check, num_cols_validity_check

In [43]:
accuracy_check = data.agg({"tx_amount": "sum"}).alias('total_tx_amount')

In [44]:
accuracy_check

sum(tx_amount)
7.642769045469997E9


In [45]:
140984588/7642769045

0.018446794240398245

In [36]:
spark.stop()